# 3 - Neural Networks 

(followed from https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html)

Neural networks can be constructed using the $\texttt{torch.nn}$ package. $\texttt{nn}$ depends on autograd to define models and differentiate them. An $\texttt{nn.Module}$ contains layers, and a method forward(input) that returns the output. It takes the input, feeds it through several layers one after the other, and then finally gives the output.

A typical training procedure for a neural network is as follows:

- Define the neural network that has some learnable parameters (or weights)
- Iterate over a dataset of inputs
- Process input through the network
- Compute the loss (how far is the output from being correct)
- Propagate gradients back into the network’s parameters
- Update the weights of the network, typically using a simple update rule: weight = weight - (learning rate) * gradient

## 3.1 - Define the Network
Let’s define this network:

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)        #nn.Conv2d(input_feature, output_feature, kernel size) The input_feature receives the nº of channels of the image
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)          # nn.Linear(in_feature, out_feature)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

#Neural Network structure resume
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


We just have defined the $\texttt{forward}$ funtion. The $\texttt{backward}$ function (where gradients are computed) is automatically defined by you using $\texttt{autograd}$. You can use any of the Tenso operations in the forward function.

The learnable parameters of a model are returned by $\texttt{net.parameters}$()

In [5]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


Let’s try a random 32x32 input

In [12]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0215, -0.0387,  0.0161,  0.1216,  0.0799,  0.1146, -0.0740,  0.1239,
          0.0271, -0.0503]], grad_fn=<AddmmBackward0>)


Zero the gradient buffers of all parameters and backprops with random gradients:

In [13]:
net.zero_grad()
out.backward(torch.randn(1, 10))

### 3.1.1 - Recap of useful definitions

- $\texttt{torch.Tensor}$ - A multi-dimensional array with support for autograd operations like $\texttt{backward}$(). Also holds the gradient with respect to (w.r.t.) the tensor.
- $\texttt{nn.Module}$ - Neural network module. Convenient way of encapsuling parameters, with helpers for moving them to GPU, exporting, loading, etc.
- $\texttt{nn.Parameter}$ - A kinf of Tensor, that automatically registered as a parameter when assigne as an attribute to a $\texttt{Module}$.
- $\texttt{autograd.Function}$ - Implements forward and backward definitions of an autograd operation. Every $\texttt{Tensor}$ operation ceates at least a single $\texttt{Function}$ node that connects to functions that created a $\texttt{Tensor}$ and encodes its history.

## 3.2 - Loss Function

A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target. There are several different loss functions under the nn package (see https://pytorch.org/docs/nn.html#loss-functions).A simple loss is: $\texttt{nn.MSELoss}$ which computes the mean-squared error between the output and the target. 

For example:

In [15]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.8505, grad_fn=<MseLossBackward0>)


Now, if you follow loss in the backward direction, using its .grad_fn attribute, you will see a graph of computations that looks like this:

So, when we call $\texttt{loss.backward}$(), the whole graph is differentiated w.r.t. the neural net parameters, and all Tensors in the graph that have $\texttt{requires_grad=True}$ will have their .$\texttt{rad}$ Tensor accumulated with the gradient.

For illustration, lets follow a few steps backward:

In [17]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

## 3.3 - Backpropagation

To backpropagate the error all we have to do is to $\texttt{loss.backward}()$. But before this, you need to clear the existing gradients, else gradients will be accumulated to existing gradients.

Now we shall call $\texttt{loss.backward}()$, and have a look at conv1’s bias gradients before and after the backward.

In [20]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([ 0.0217, -0.0114, -0.0102, -0.0043,  0.0096,  0.0071])


Now, we have seen how to use loss functions.

$\textbf{Useful:}$: The neural network package contains various modules and loss functions that form the building blocks of deep neural networks. A full list with documentation is here -> https://pytorch.org/docs/nn

## 3.4 - Update the weights

The simplest update rule used in practice is the Stochastic Gradient Descent (SGD):

We can implement this using simple Python code:

In [30]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [32]:
#net.parameters is:
net.parameters

<bound method Module.parameters of Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)>

However, as you use neural networks, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc. To enable this, we built a small package: $\texttt{torch.optim}$ that implements all these methods. Using it is very simple:

In [33]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update